In [36]:
import numpy as np
import pandas as pd
import openpyxl


########################################part 1#####################################

# Lese die Excel-Datei mit dem Profil, welches varriert werden soll ein
#df_profil = pd.read_excel("profil_spring.xlsx")
#df_profil = pd.read_excel("profil_summer.xlsx")
#df_profil = pd.read_excel("profil_fall.xlsx")
df_profil = pd.read_excel("profil_winter.xlsx")

# Erstelle ein leeres Pandas-DataFrame für die Ergebnisse
results = pd.DataFrame()

# Für jedes Szenario:   
for i in range(1000):
    # Erstelle ein Array mit normalverteilten Zufallsvariablen für jeden Bus und Jahreszeit
    random_variables = np.random.normal(0, 0.010, (len(df_profil), 1))
    
    # Berechne die Summe der Zufallsvariablen
    sum_random_variables = sum(random_variables)
       
    # Berechne den Faktor, um den jede Zufallsvariable angepasst werden muss, damit die Summe 0 ergibt
    factor = -sum_random_variables / len(random_variables)

    # Passe jede Zufallsvariable an, indem der Faktor multipliziert wird
    random_variables_adjusted = random_variables + factor

    # Die gewünschte Toleranz weil die Berechnung von Gleitkommazahlen ungenau ist und somit die Summe der angepassten Zufallszahlen minimal von 0 abweichen kann
    tolerance = 1e-10

    # Prüfe, ob die Summe der angepassten Zufallsvariablen innerhalb der Toleranz ist
    assert np.allclose(np.sum(random_variables_adjusted, axis=0), 0, atol=tolerance)
    
    # Füge die Zufallsvariablen den profilparametern hinzu
    df_varied = df_profil.iloc[:, 1:] + random_variables_adjusted

    # Setze die Spaltenüberschriften auf "Scenario i"
    df_varied.columns = ["S{}".format(i+1)]
    
    # Füge die veränderten Lastparameter dem DataFrame der Ergebnisse hinzu
    results = pd.concat([results, df_varied], axis=1)
#    results_T = results.T    
      
# Speichere das DataFrame der Ergebnisse in einer Excel-Datei
#results.to_excel("Ergebnisse_2.xlsx", index=False)
#results_T.to_excel("Ergebnisse.xlsx", index=False)
#results.head()

########################################part 2#####################################
#load_bus = pd.read_excel('load_bus_spring.xlsx')
#load_bus = pd.read_excel('load_bus_summer.xlsx')
#load_bus = pd.read_excel('load_bus_fall.xlsx')
load_bus = pd.read_excel('load_bus_winter.xlsx')

load_profil = pd.DataFrame(columns=['b'] + ['t'+str(i) for i in range(1,97)])
load_profil['b'] = load_bus['b']
for i in range(1,97):
    load_profil['t'+str(i)] = df_profil['profil'][i-1] * load_bus['Active demand']
    
########################################part 3#####################################

# Berechne die Lastabweichungen zum normalen Profil df_profil
load_variation = pd.DataFrame()

# Iteration über alle Szenarien
for scenario in results.columns:
    results[scenario] = results[scenario].fillna(0)
    load_variation_scenario = pd.DataFrame(columns=['Scenario','b'] + ['t'+ str(i) for i in range(1,97)])
    load_variation_scenario['Scenario'] = [scenario]*33
    load_variation_scenario['b'] = load_bus['b']
    for i in range(1,97):
        load_variation_scenario['t'+ str(i)] = results[scenario][i-1] * load_bus['Active demand'] - load_profil['t'+str(i)]
#        load_variation_scenario.subtract(load_profil)
    load_variation = pd.concat([load_variation,load_variation_scenario], ignore_index=True )
    
# umbennenung für Kompatibilität mit Gams   
load_variation.rename(columns={'t'+str(i): str(i) for i in range(1,97)}, inplace=True)
    
#load_variation.head()
load_variation.to_excel("Data.xlsx", index=False)